## Drill Hole Index Finder

Here we will find the index numbers(rows) of the Drill Holes (DH's) that we need to extract from the **sarig_rs_chem_exp.csv** dataset corresponding to the boreholes in the **sarig_dh_details_exp** dataset.

In [1]:
import pandas as pd
import numpy as np
import itertools
import os
import pickle

pd.set_option('display.max_columns', None)

In [2]:
os.chdir('..')
os.getcwd()

'H:\\ExploreSA'

##### Loading the DF that we created from the previous notebook and extracting the unique Drill Hole numbers. We can then extract the Geochemical Assay results for these boreholes from the "sarig_rs_chem_exp.csv" dataset.

In [3]:
#load Gold data
df_gold = pd.read_csv('Cleaned_datasets/sarig_dh_details_Gold.csv')
df_gold.head()

C:\Users\Russell\Anaconda3\envs\PythonGPU\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,DRILLHOLE_NO,DH_NAME,DH_OTHER_NAME,PACE_DH,PACE_ROUND_NO,REPRESENTATIVE_DH,REPRESENTATIVE_DH_COMMENTS,DH_UNIT_NO,MAX_DRILLED_DEPTH,MAX_DRILLED_DEPTH_DATE,CORED_LENGTH,TENEMENT,OPERATOR_CODE,OPERATOR_NAME,TARGET_COMMODITIES,MINERAL_CLASS,PETROLEUM_CLASS,STRATIGRAPHIC_CLASS,ENGINEERING_CLASS,SEISMIC_POINT_CLASS,WATER_WELL_CLASS,WATER_POINT_CLASS,DRILLING_METHODS,STRAT_LOG,LITHO_LOG,PETROPHYSICAL_LOG,GEOCHEMISTRY,PETROLOGY,BIOSTRATIGRAPHY,SPECTRAL_SCANNED,CORE_LIBRARY,REFERENCES,HISTORICAL_DOCUMENTS,COMMENTS,MAP_250000,MAP_100000,MAP_50K_NO,SITE_NO,EASTING_GDA2020,NORTHING_GDA2020,ZONE_GDA2020,LONGITUDE_GDA2020,LATITUDE_GDA2020,LONGITUDE_GDA94,LATITUDE_GDA94,HORIZ_ACCRCY_M,ELEVATION_M,INCLINATION,AZIMUTH,SURVEY_METHOD_CODE,SURVEY_METHOD
0,61638,BH34 34,NaN,N,NaN,N,1 OF 35 AUGER HOLES IN TAILINGS.,6628 14669,144.2,2017-06-17,0.0,,SDM,South Australia. Department of Mines and Energy.,"Gold, Copper",Y,N,N,N,N,N,N,Auger (Hand),N,N,N,N,N,N,N,Y,Y,Y,Tailings.,SI5409 ADELAIDE,6628 Adelaide,2,325873,308944.99,6129703.52,54,138.907534,-34.957203,138.907524,-34.957189,10.0,444.35,-90.0,NaN,GPSUN,GPS type unknown
1,147530,STO 1,NaN,N,NaN,N,NaN,6729 1506,155.5,1995-02-14,NaN,,SDM,South Australia. Department of Mines and Energy.,"Gold, Base Metals, Metalliferous Minerals",Y,N,N,N,N,N,N,Reverse Circulation,N,Y,Y,Y,Y,N,N,Y,Y,N,NaN,SI5409 ADELAIDE,6729 Eudunda,2,195110,344854.61,6204552.69,54,139.314355,-34.288648,139.314345,-34.288633,20.0,102.00,-90.0,NaN,GPS,Global Positioning System
2,147531,STO 2,NaN,N,NaN,N,NaN,6729 1507,94.0,1995-02-15,NaN,,SDM,South Australia. Department of Mines and Energy.,"Gold, Base Metals, Metalliferous Minerals",Y,N,N,N,N,N,N,Reverse Circulation,N,Y,Y,Y,N,N,N,Y,Y,Y,"BLOCKED OFF IN SANDS TERMINATED HOLE, Traverse...",SI5409 ADELAIDE,6729 Eudunda,2,195111,346316.55,6204575.59,54,139.330237,-34.288659,139.330227,-34.288644,20.0,78.00,-90.0,NaN,GPS,Global Positioning System
3,147532,STO 3,NaN,N,NaN,N,NaN,6729 1508,66.1,1995-02-16,NaN,,SDM,South Australia. Department of Mines and Energy.,"Gold, Base Metals, Metalliferous Minerals",Y,N,N,N,N,N,N,Reverse Circulation,N,Y,Y,Y,N,N,N,Y,Y,N,NaN,SI5409 ADELAIDE,6729 Eudunda,2,195112,347322.45,6204592.72,54,139.341165,-34.288653,139.341155,-34.288638,20.0,66.00,-90.0,NaN,GPS,Global Positioning System
4,147533,STO 4,NaN,N,NaN,N,NaN,6729 1509,126.0,1995-02-17,NaN,,SDM,South Australia. Department of Mines and Energy.,"Gold, Base Metals, Metalliferous Minerals",Y,N,N,N,N,N,N,Reverse Circulation,N,Y,Y,Y,N,Y,N,Y,Y,Y,"LOST CIRCULATION ABANDONDED HOLE, Traverse: T1...",SI5409 ADELAIDE,6729 Eudunda,2,195113,349603.60,6204634.65,54,139.365948,-34.288607,139.365938,-34.288593,20.0,55.00,-90.0,NaN,GPS,Global Positioning System


In [4]:
# Get unique drill hole values and get the total number
dh_nos_gold = df_gold.DRILLHOLE_NO.unique()
len(dh_nos_gold)

41979

### Chunking - Processing Large CSV files

The **sarig_rs_chem_exp.csv** is a 10GB CSV file. Loading the entire CSV into a pandas DF is not possible unless you have 32GB+ RAM. As most people will be doing this on laptops with lower memory, we can use Panda's Chunking technique to load the file in chunks and process each chunk individually.

Our aim is to extract assay information for only those boreholes that exist in our **sarig_dh_details_Gold.csv** dataset.

In [5]:
df_iter = pd.read_csv('SARIG_Data_Package/sarig_rs_chem_exp.csv', chunksize=200000)

dh_nos_list = []

for chunk_no, chunk in enumerate(df_iter):
    
    print(chunk_no)
    dh_nos = chunk.DRILLHOLE_NUMBER.values
    dh_nos_list.append(dh_nos)

C:\Users\Russell\Anaconda3\envs\PythonGPU\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (6,10,13,28,30,34,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0


C:\Users\Russell\Anaconda3\envs\PythonGPU\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,16,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1


C:\Users\Russell\Anaconda3\envs\PythonGPU\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,13,16,28,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


...

140
141
142
143
144


C:\Users\Russell\Anaconda3\envs\PythonGPU\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3,4,5,7,8,9,11,13,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


145
146
147
148
149


C:\Users\Russell\Anaconda3\envs\PythonGPU\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3,4,5,7,13,28,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


150
151
152


In [6]:
# Flattening Lists of DH nos from individual chunks into one list

import itertools

flattened_list  = list(itertools.chain(*dh_nos_list))

In [7]:
#Checking the lengths of both our lists

len(dh_nos_gold), len(flattened_list)

(41979, 30445428)

#### Matching Drill holes from *dh_nos_gold* list to the ones in our *flattened_list* and getting the index number

Wells from one list can be easily matched with another using loops or list comprehension. Our flattened list contains 30M values. Matching 41k individual values to 30M values can take days. We can solve this problem in seconds using a nifty hack - Table Joins, and it only takes 10 seconds!


In [8]:
#Creating a DF from our lists
df_dh_nos_gold = pd.DataFrame(dh_nos_gold, columns=['dh_gold_name'])
df_dh_nos_gold.head()

,dh_gold_name
0,61638
1,147530
2,147531
3,147532
4,147533


In [9]:
df_dh_nos_flat_list = pd.DataFrame(flattened_list, columns=['dh_gold_flat_list'])
df_dh_nos_flat_list.head()

,dh_gold_flat_list
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


We can merge *df_dh_nos_flat_list* with *df_dh_nos_gold* using a left join.

In [10]:
merged_left = pd.merge(left=df_dh_nos_flat_list, right=df_dh_nos_gold, how='left', left_on='dh_gold_flat_list', right_on='dh_gold_name')

Lets look at a quick sample of our merged dataset

In [11]:
merged_left.sample(30)

,dh_gold_flat_list,dh_gold_name
7962877,208084.0,208084.0
9487681,312954.0,312954.0
21025391,291003.0,NaN
2449982,293124.0,293124.0
12774522,312914.0,312914.0
3692404,293419.0,293419.0
2950280,293372.0,293372.0
20326093,272986.0,272986.0
4826356,293540.0,293540.0
29042667,269336.0,269336.0


In [12]:
#Dropping all columns with NULL values to get reqired drill holes
merged_left_dropna = merged_left.dropna()

In [13]:
#another sample after dropping nulls
merged_left_dropna.sample(10)

,dh_gold_flat_list,dh_gold_name
12951079,312838.0,312838.0
11260316,226405.0,226405.0
13261399,313033.0,313033.0
13507941,313013.0,313013.0
6822739,332833.0,332833.0
12070060,312968.0,312968.0
8948878,265066.0,265066.0
26999827,332810.0,332810.0
9173671,276160.0,276160.0
23274257,332880.0,332880.0


In [14]:
#Extracting index of drill holes into list
final_index_dh_gold =  merged_left_dropna.index.values

In [18]:
#Saving the list to disk for later use
with open('Cleaned_datasets/final_index_dh_gold.pkl', 'wb') as f:
    pickle.dump(final_index_dh_gold, f)

In [15]:
#Number of rows we will need to extract from Assay DF
len(final_index_dh_gold)

19552741

In [17]:
#preview of the first 100 index values in the list

final_index_dh_gold[:100]

array([  14,   37,   38,   39,   40,   41,   42,   43,   44,   45,   46,
         47,   48,   96,   97,   98, 2348, 2349, 2361, 2373, 2385, 2397,
       2398, 2399, 2411, 2412, 2413, 2414, 2415, 2416, 2417, 2429, 2430,
       2431, 2432, 2444, 2445, 2446, 2447, 2448, 2449, 2450, 2451, 2452,
       2453, 2454, 2455, 2456, 2457, 2458, 2459, 2460, 2461, 2548, 2549,
       2947, 2948, 2950, 2951, 2952, 2953, 2954, 2955, 2956, 2957, 2958,
       2959, 2960, 2961, 3806, 3807, 3808, 3809, 3810, 3811, 3812, 3813,
       3814, 3815, 3816, 3817, 3818, 3819, 3820, 3821, 3822, 3823, 3824,
       3825, 3826, 3827, 3828, 3829, 3830, 3831, 4711, 4712, 4713, 4714,
       4715], dtype=int64)